<span type="title">Spring MVC</span> | <span type="update">2018-10-25</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍了 Spring MVC 的基本使用。</p></span>

# Spring MVC 概述

Spring MVC 是一套用来代替 Struts 的 Servlet 框架。相比较传统的 Servlet 技术，基于 Struts 的框架设计模式采用一个 Super Servlet 对所有的 URL 请求进行转发和分派，然后再交给子 Servlet 进行请求，之后由 JSP 等模板技术完成渲染工作。这套框架很好的避免了大工程需要繁琐的配置 web.xml 文件、同时能够对于 URL 请求进行更轻松的解析和处理。Spring MVC 基于 Spring IOC 容器，是 Struts 的替代产品，主要用来进行 Controller 层的控制，完成 View 层和 Model 层之间的交互。

其基本作用原理是：使用 Spring 内置的 Servlet 拦截所有请求，交给自家的 Handler 完成处理，之后再委托 JSP 等模板引擎进行渲染。需要配置 web.xml 文件完成拦截和监听，配置 spring-mvc.xml 等配置文件来具体定义 Handler 的查找路径、模板引擎的处理器等等。

Spring MVC 相比较 Struts 的优点在于：天生和 Spring 框架集成，支持 Restful 风格的开发，从 URL 到映射的处理十分方便，数据验证、格式化和绑定功能强大、好用，对于异常的处理支持较好，对于静态资源文件提供了支持。



## MVC 使用概览

**WEB.XML配置**

Web.xml 文件需要配置如下的 Servlet，完成拦截，需要指明类为 DispatcherServlet，此外，需要传入 Spring MVC 的 xml 格式配置，放置在 Servlet 的上下文中，名称为 ContextConfigLocation，如果这里没有配置，则默认寻找 `/WEB-INF/{serlvet-name}-servlet.xml` 文件作为配置。

```xml
<servlet>
    <servlet-name>springDispatcherServlet</servlet-name>
    <servlet-class>org.springframework.web.servlet.DispatcherServlet</servlet-class>
    <init-param>
        <param-name>contextConfigLocation</param-name>
        <param-value>springmvc.cfg.xml</param-value>
    </init-param>
</servlet>
<servlet-mapping>
    <servlet-name>springDispatcherServlet</servlet-name>
    <url-pattern>/</url-pattern>
</servlet-mapping>
```

为了支持 Restful 风格 URL 格式，推荐添加如下过滤器, HiddenHttpMethodFilter 过滤器可以对请求进行过滤，根据隐藏字段，将 POST, GET 请求转换成 POST, GET, PUT, DELETE 四种原生的 HTTP 方法。

```xml
<filter>
    <filter-name>methodFilter</filter-name>
    <filter-class>org.springframework.web.filter.HiddenHttpMethodFilter</filter-class>
</filter>
<filter-mapping>
    <filter-name>methodFilter</filter-name>
    <url-pattern>/*</url-pattern>
</filter-mapping>
```

Ps. 使用 IDEA 生成的 web.xml 有时候会莫名其妙的导致 404，无法找到 Spring Servlet，但是 jsp 页面可以直接访问（所以肯定是 web.xml 的锅，但是这个文件确实能够找到，没有报找不到 web.xml 的错误，但就是解析不了里面的内容），这时候新建一个 web.xml，然后将之前设置复制进来即可。

PPs. 这里的 Spring Servlet 的 URL 是 `/` 最好，而不要是 `/*`, 否则将会自动处理任意 .jsp 文件，当然，如果你在 SPRING-MVC 的配置文件中指定了如果处理不了，交还应用服务器进行处理这一选项 `default-serlvet` 的话，怎样写无所谓。

**SPRING-SERVLET.XML配置**

一个基本的 springmvc.cfg.xml 配置文件如下：

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
       xmlns:beans="http://www.springframework.org/schema/beans"
       xmlns:context="http://www.springframework.org/schema/context"
       xmlns:mvc="http://www.springframework.org/schema/mvc"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/context
       http://www.springframework.org/schema/context/spring-context.xsd
       http://www.springframework.org/schema/mvc
       http://www.springframework.org/schema/mvc/spring-mvc.xsd
">
    <context:component-scan base-package="com.mazhangjing.mvc.hello"/>
    <mvc:annotation-driven />
    <bean class="org.springframework.web.servlet.view.InternalResourceViewResolver" 
          p:prefix="/WEB-INF/views/" p:suffix=".jsp"/>

    <!--或者可以不经过 Handler，直接完成 View-Controller 和 View 的对应。此种情况下需要添加 mvc 注解驱动-->
    <mvc:view-controller path="/success" view-name="success" />

</beans>
```

注意这里面较为重要的一些设置：

- 【基于注解扫描的 Handler】：启用 context:component-scan 和 mvc:annotation-driven, 这两个选项能够保证指定位置的包中的注解能够被被配置为 bean，并且能够作为 Handler 进行映射。

- 【ViewResolver 页面解析器】：一般使用的是 InternalResourceViewResolver，传入 suffix 配置 .jsp 作为后缀，perfix 作为前缀，之后，你可以在 Handler 中返回字符串，然后 ViewResolver 会自动进行前缀+字符串+后缀的拼接，然后找到指定位置的 jsp 并进行解析。

    - 尤其需要注意，此处的 perfix 前缀需要以 `/` 结尾，否则拼接出来的地址会是错误的！！！

    - 除了使用 Handler 进行映射，也可以直接使用 mvc:view-controller 映射指定 URL 到指定的 view，这里的 view-name 会拼接 ViewResolver 找到对应位置的视图引擎自动解析并且返回 HttpResponse 流。

**CONTROLLER HANDLER 配置**

```java
@Controller
public class HelloWorld {
    @RequestMapping(value = "/helloworld",method = RequestMethod.GET)
    public String sayHello() {
        System.out.println("Controller get sayHello request from SpringMVC Servlet");
        return "success";
    }
}
```

为了让 Spring 添加 bean，需要添加 @Controller 注解，为了指定映射的 URL，需要指定类或者方法的 @RequestMapping 注解，提供 URL 和处理的 Http 方法即可。签名随意，返回值为 String 会自动被视图解析器拼接，找到视图层，进行返回。之后访问 `localhost:port/helloworld` 即可调用此处的 Handler 进行处理。

**Spring MVC 具体工作流程**

1、【找到 DispatcherServlet】

  启动 TOMCAT 后会自动在 web.xml 找到 Servlet 进行请求响应。而在 XML 中，我们使用了 Spring 内置的 servlet
  处理所有的发送请求，这个 servlet class 是 org.springframework.web.servlet.DispatcherServlet. 相当于 Structs 的
  巨型 Dispatcher。此 Servlet 默认会处理 WBE-INF 下的 servlet_name + "-servlet.xml" 结尾的描述文件，用作配置。此外，
  也可以传递 configLocation 参数为指定 classpath 或者 文件路径下的 xml Spring 配置文件。
 
2、【找到 HandlerMapping】
  
  Spring 配置文件规定了这个 Servlet 如何处理相应。区别于直接在 web.xml 写 URL 转发规则， Spring 使用注解来决定
  转发，因此启用 component-scan 注解扫描，Spring 会自动扫描 @Controller 注解类型 以及 其内部的 @RequestMapping 方法
  用来进行转发，@RequestMapping 需要提供 value 和 method 以声明自身可以处理何种转发。
 
  1-2步即是 TOMCAT 找到 Spring Dispatcher ， 然后 SD 通过 xml 配置找到匹配注解的 RequestMapping 方法进行转发。这里完成了
  传统 Servlet 从 URL 到 Java 类方法的过程。
 
3、【找到 ViewResolver】
  
  接下来可以直接像 Servlet 一样分派一个 JSP 处理 URL 请求，这里使用的是一个更好玩的类，叫做 org.springframework.web.
  servlet.view.InternalResourceViewResolver。这个 ViewResolver 在 Spring xml 中进行了声明，提供了 prefix 和 suffix
  属性，这里不通过交给 jsp 进行分派，而是返回一个 String，因此交给了此 ViewResolver 获得指定 prefix + String value + suffix
  位置的 jsp 文件进行自动。最后由此处的 jsp 处理视图相应。
  这里的好处是，可以通过替换 ViewResolver 来达到前后端分离，或直接处理前端内容，更加灵活。此外，对于超级多的 jsp 页面逻辑支持也更好。
 

## RequestMapping 注解

RequestMapping 用来解析 URL 到指定的 Handler，除了修饰方法以外，还可以修饰类。使用基于类 + 基于方法的 RequestMapping 可以用来分级的解析地址：`/springmvc/abc` 。这样的话，类注解对应高一级的地址层级，而方法注解对应稍低一级的地址层级，这样比直接在 web.xml 根据 * 和 后缀过滤  URL 方便很多。

```java
@Controller
@RequestMapping("/springmvc")
public class MVCTest {
    private static final String SUCCESS = "success";
    @RequestMapping("/abc") public String testMapping() { return SUCCESS; }
}
```
**过滤方法、地址匹配**

可以指定 Mapping 的 method 参数指定解析的 HTTP 方法，如果没有对应的方法被 Handle，则报错。

可以通过星号通配符匹配任意字符：其中 `*` 表示任意字符（单层路径）， `？` 表示一个字符， `**` 表示多层路径

```java
@RequestMapping(value = "/testMethod",method = RequestMethod.POST)
public String testMethod() { return SUCCESS; }
@RequestMapping("/testAnt*") public String testAnt() { return SUCCESS; }
```

**过滤URL参数、请求头**

可以通过 params 指定 URL 参数的匹配，可选 有/无某参数，参数等于、不等于某值 这四种情况。

可以通过 headers 指定针对仅含有某请求头匹配情况进行相应，比如下面仅对 keep-alive 的 Connection 的 headers，URL 参数包含 ursename，passwd，school，并且 passwd 不等于 10，school 必须等于 ccnu，没有 superuser 这个参数的 URL 进行相应，比如: `testParameter/?username=tom&passwd=233&school=ccnu&nosuperuser`

```java
@RequestMapping(
        value = "testParameter",
        params = {"username","passwd!=10","school=ccnu","!superuser"},
        headers = {"Connection: keep-alive"}
)
public String testParam() {
    return SUCCESS;
}
```

**从 URL 获取变量**

我们可以从 URL 中获得某些数据，而不是从 URL 的参数中获取：

```java
@RequestMapping("/testPathVar/{id}")
public String testPathVar(@PathVariable("id") Integer id) { System.out.println("id is " + id); return SUCCESS; }
```

这样的话，id 会被作为 String 从 URL 上匹配并且获取，然后传递给签名入参的 Interger id（进行了类型自动转换），然后获取了此 URL 上的信息。

为了做到从 URL 上提取数据，作为入参传递，必须指定此入参为 `@PathVariable('name_in_url')`，传入的参数为其在 URL 上双括号包裹起来的名称。注意，这种情况下，`/testPathVar/` 将不会匹配此条记录。

**RESTFUL URL 支持**

PathVariable 为 RESTFUL 风格的 URL 提供了支持，具体如下：RESTFUL 风格通过 URL + HTTP METHOD 达到 CRUD 的效果

 * 新增： /order POST   add
 * 修改： /order/1 PUT  update?id=1
 * 获取： /order/1 GET  get?id=1
 * 删除： /order/1 DELETE  delete?id=1
 
因为浏览器一般不支持 PUT 和 DELETE，因此需要配置 Servlet Listener，并且对于所有 URL 请求都启用`org.springframework.web.filter.HiddenHttpMethodFilter` 以将带有隐含域的 POST 请求转换为 DELETE 和 PUT，具体做法如下：

对于 PUT 和 delete 请求，在 form 需要添加一个 hidden type 的 name 为 _method 的输入，value 为 DELETE/PUT。

具体的 HTML 为：

```html
POST 新建对象: 
<form method="post" action="/springmvc/testRest">
    <input type="submit" value="post it" />
</form>
GET 获取某对象: 
<h1><a href="/springmvc/testRest/1"> Test REST Get </a> </h1>
PUT 更新某对象:
<form method="post" action="/springmvc/testRest/1">
    <input type="hidden" name="_method" value="PUT"/>
    <input type="submit" value="put it" />
</form>
DELETE 删除某对象:
<form method="post" action="/springmvc/testRest/1">
    <input type="hidden" name="_method" value="DELETE"/>
    <input type="submit" value="delete it" />
</form>
```
此外，PUT 和 DELETE 在 Dispatcher Mapping 中需要指定 method = `RequestMethod.PUT/DELETE`. 对于 get、put、delete 需要启用 {} 和 @PathVariable 注解。

```java
@RequestMapping(value = "/testRest/{id}",method = RequestMethod.GET)
public String testRestGET(@PathVariable Integer id) {
    System.out.println("rest GET id " + id);
    return SUCCESS;
}

@RequestMapping(value = "/testRest",method = RequestMethod.POST)
public String testRestPOST() {
    System.out.println("rest POST");
    return SUCCESS;
}

@RequestMapping(value = "/testRest/{id}",method = RequestMethod.DELETE)
public String testRestDEL(@PathVariable Integer id) {
    System.out.println("rest DEL id " + id);
    return SUCCESS;
}

@RequestMapping(value = "/testRest/{id}",method = RequestMethod.PUT)
public String testRestPUT(@PathVariable Integer id) {
    System.out.println("rest PUT id " + id);
    return SUCCESS;
}
```

**获取参数、Header、Cookie**

通过设置 RequestParam 来要求输入某一参数信息或者设置默认值、获取此参数信息。

通过设置 RequestHeader 来获取头部信息

通过设置 CookieValue 来绑定/获取 Cookie 值。

有时候可能没有对应的值或者记录，可以使用 required 来设置是否必须有此值/记录才能解析，使用 defaultValue 来设置如果没有，且 required = false 的情况下使用的替换值。

需要注意，获取的值必须和入参参数的类型匹配，或者起码可以转换，如果不能转换，则报错。

```java
@RequestMapping(value = "/testPara")
public String testPara(
        @RequestParam(value = "username", required = false, defaultValue = "Guest") String username,
        @RequestHeader(value = "Accept-Language", required = false) String al,
        @CookieValue(value = "JSESSIONID",required = false) String sessionId) {
    System.out.println("username is " + username);
    System.out.println("Accept language is " +al);
    System.out.println("session id is " +sessionId);
    return SUCCESS;
}
```

**获取 Servlet 原生 API**

下面展示了入参使用传统 ServletAPI 的情景，将这些类型放置在入参处即可，不需要加注解。Spring MVC 具体支持 HSRequest/Response、Session、Input/OutputStream、Reader/Writer、Principal 这些原生 API 作为入参传递。

注意，返回值可以不为 String，不交给视图解析器，而是直接向输出 write 数据。

```java
@RequestMapping("/testServlet")
public void testServletAPI(HttpServletRequest request, HttpServletResponse response,
                             Writer out) throws IOException {
    System.out.println(request + "" +  response);
    out.write("Hello from Servlet API");
}
```

## ModelAndView 对象

**手动返回 ModelAndView 对象传递 RequestAttr**

一般来说，通过 String 返回找到 view 层即可完成响应。但是，如果需要向 view 层写数据，就需要使用 modelAndView 对象作为返回。返回 String 则视图解析器返回视图 View，也可以返回 ModelAndView 对象，此对象包括了模型和 View 层信息，可以向 View 层传递数据。区别于直接返回 String 并直接返回 view 层，视图解析器将会将 modelAndView 对象中找到对应的 view 层，将 model 的键值对解析到 view 层的 rS 域中，然后再返回 view 层。


```java
//Controller.java
@RequestMapping("/testModelAndView") public ModelAndView testModelAndView() {
    String viewName = SUCCESS;
    ModelAndView modelAndView = new ModelAndView(viewName);
    modelAndView.addObject("time",new Date());
    return modelAndView;
}
//success.jsp
${requestScope.time}
```

如上所示，我们可以在 ModelAndView 对象中添加 key-value，然后这些 key-value 会被自动 set 到 Request Attribute 中，在 View 层可以直接取出。

实际上，所有直接返回 String，通过 ViewResolver 找到 JSP 进行解析的例子都默认自动构建了 ModelAndView，然后从此对象获得 view 层，将 Request 和 Session 等对象传递过去完成渲染工作。这个步骤是 Spring MVC 自动完成的，当然，我们也可以手动完成。需要注意，手动返回 MAV 的时候，需要设置到何处找到 view 层的信息（一般通过构造器直接传入）。

**使用 Map 入参传递 RequestAttr 数据**

MAV 方式过于手动且繁琐，实际情况下有一种简写方式，可以使用 Map 作为入参，然后在 Map 中放置键值对，然后从 rS 域或者直接取出。入参除了 Map 类型，还可以使用 ModelMap、Model 类型。

这种方式其实是自动的将 Map 的数据放置到了 ModelAndView 的 ModelMap 中，然后通过处理 ModelAndView 进行的处理。视图解析器对 ModelAndView 的处理方法是：通过将 MAV 中 ModelMap 的 k-v 对 设置为请求域的属性，然后去解析 View 层，将 HTML 作为结果返回。

```java
@RequestMapping("/testMap") public String testMap(Map<String, Object> map) {
    map.put("names", Arrays.asList("A","B","C"));
    return SUCCESS;
}
//success.jsp
${requestScope.names}
```

**将 RequestAttr 的 K-V对 同时添加到 SessionAttr 上**

## SessionAttribute 注解

放置在 Map 入参处的值，可以通过 rs 或者 key 直接取出。通过设置：@SessionAttributes({"user"}) 可以同时放置到 SessionAttributes 域中。此域的 value 属性仅允许设置过的 key 保存，如果没有设置，则不允许。此域的 type 属性可以允许按照 class 类型来讲 rs 域值放置到 sa 中。比如：@SessionAttributes(type={String.class}) 这样将会让所有 rs 域中 String 类型的数据放置到 SessionAttribute 上。

```java
@SessionAttributes(value = {"user"},types = {String.class})
@Controller
@RequestMapping("/springmvc")
public class MVCTest {
    ...
    @RequestMapping("/testSession")
    public String testSessionAttribute(Map<String, Object> map) {
        User user = new User(1,"corkine","123456","c@cm.cn",15);
        map.put("user",user);
        return SUCCESS;
    }
```

## ModelAttribute 注解

**从 FORM 到 POJO**

SpringMVC 支持直接 POST 参数到一个 POJO 中，还支持级联属性。以下是两个 POJO 类：

```java
public class User {
    private Integer id;
    private String name;
    private String password;
    private String email;
    private Integer age;
    private Address address;
    ...
}
public class Address {
    private String province;
    private String city;
    ...
}
```

注意，POJO 必须提供无参构造器和 get/set 方法。

以下是 HTML 调用，需要使用 post 方法，此外，form 的 name 需要对应 pojo 的属性名称， pojo 需要提供 set 方法，支持级联属性，比如 address.city 表示 address 对象 的 city 属性。直接在 name 中设置  'address.city' 即可。如果获取不到则设置为 null；

```html
/springmvc/testPOJO
<form method="post" action="">
    username : <input type="text" name="name" /><br>
    password : <input type="password" name="password" /><br>
    email : <input type="text" name="email" /><br>
    age : <input type="text" name="age" /><br>
    province : <input type="text" name="address.province" /><br>
    city : <input type="text" name="address.city" /><br>
    <input type="submit" value="Submit" />
</form>
```

之后直接在入参处写一个此类型的入参即可调用获得构造好的对象。注意，不要写 @RequestVariable，直接 POST 即可。这一套从 HTML 到 Bean 的方式类似于 Servlet 的 Stand Action Use-Bean。

```java
@RequestMapping("/testPOJO")
public String testPojo(@ModelAttribute("user") User user) { System.out.println(user); return SUCCESS; }
```

这里获取到的 user 会被同样放置到 requestScope 中，通过入参的 @ModelAttribute(value) 设置 requestScope 中获取此 user 的方式，比如 @ModelAttribute（value = "user"） 在  JSP 渲染页面的 ${requestScope.user} 通过这种方式获取此对象。如果不提供此注解，则默认以类名第一个字母小写作为变量放置到 requestScope 中。

```xml
/success.jsp
requestScope user: ${requestScope.user}
```

很显然的，Spring MVC 完成了从 FORM 到 POJO 的工作，那么它是如何完成的呢？

在上面我们说过，ModelAndView 对象负责用来传递 RequestAttribute 属性和 SessionAttribute 属性，并且自动完成 View 层的查找和交接工作。

这里的过程分为两步，其一是从 FORM 表单提交到构建 User 对象，到作为入参的 User 对象引用，其二是作为入参的 User 对象到 ModelAndView 对象的 ModelMap 集合，然后到 RequestAttribute 属性，以至于在 View 层能够从 requestScope 获取此对象。

**从数据库+表单到 POJO**

在一些情况下，我们并不想让表单填写所有的字段，有一些字段我们希望从数据库加载，然后只提供有限的字段以供表单修改，然后将其合并成为 POJO 作为入参传递到 Controller 的对应 Mapping 的方法中的入参中。为做到这一点，我们需要添加一个带有 @ModelAttribute 注解的方法，以完成从数据库到 FORM 调用前这一段的 User 对象的构建。

从数据库获取记录首先要从表单找到某个字段，作为查找的基础进行查找，然后获取到此记录后，再将表单剩余的字段更新此对象，最后作为入参进行设置。因此，这里需要提供一个用作查找数据的入参。找到数据后，需要将数据传递出去，这里使用的是 Map 的入参，根据前面的介绍，我们知道这里的含义是创建一个 ModelAndView，然后填入 ModelMap 的一个字段的意思。

之后，SpringMVC 会自动进行此对象根据表单的更新，然后再 Controller 的对应 Mapping 方法的入参中，使用 @ModelAttribute 注解按照 key 的名称取出即可。

一般来说，我们使用类名第一个字母小写作为 key 的名称，那样的话，这里的入参其实可以省略 @ModelAndView 注解。

```java
@ModelAttribute
public void getUser(@RequestParam(value = "id", required = false) Integer id,
                    Map<String, Object> map) {
    if (id != null) {
        User user = new User(1,"tom","123456","tom@tom.com",12);
        System.out.println("Get" + user);
        map.put("form_user",user);
    }
}
@RequestMapping("/testModelAttribute")
public String testModelAttribute(@ModelAttribute(value = "form_user") User user) {
    System.out.println("Edit " + user);
    return SUCCESS;
}
```

**内部原理详解**

具体的 ModelAttribute 调用方法如下：

首先，当 form 进行提交的时候，首先创建了一个叫做 WebDataBinder 的对象，此对象具有两个关键属性，target 和 objectName。target
用来获取模型现有对象，作为数据的入口，objName 用来指定一个字符串，用来作为更新表单数据后的数据出口标识符。

首先要获取模型数据，具体而言，先找到有 @ModelAttribute 注解的方法，将 FORM 指定字段作为参数传入，执行这段代码以更新入参的 Map 对象，并且将其放置在一个叫做 implicitModel 对象中。然后从 implicitModel 中查找对应的属性值，如果存在，ok。如果不存在，则验证当前 Handler 是否采用了 @SessionAttributes 修饰，然后从中寻找 attrName 对应的属性值，如果有此修饰符，并且有 key，但是获取不到 value，则抛出异常。如果没有此修饰符，则通过反射创建 POJO 对象。

数据的辨识名称首先如果在 Mapping 的入参 User 前没有 @ModelAttribute 修饰的话，那么我们使用类的第一个字母小写。如果有修饰的话，
使用 value 值作为 ObjName。

当得到模型的数据后，现在的 target 已经从 @ModelAttribute 的 implicitModel 中通过数据辨识名称或者 @SessionAttribute 中通关数据辨识名称获取到对象了。然后从 form 获取字段，复写 target 对象。完成之后，将数据重新传回给 implicitModel 对应字段，进而传递给 requestScope 中。而把 target 作为参数传递给目标方法的入参（Mapping 方法的 User user）。

注意，如果在 Mapping 中规定的数据标识符必须可以和 数据库取出后放置到 Map 的 Key 对上。否则的话，会创建新的 POJO 对象。

注意，如果启用了 @SessionAttribute 但是在 @MA 无法找到，在 @SA 找到但是无值 的话，抛出异常而不是新建 POJO 对象。

注意，通过 ModelAttribute 方法创建的对象最后会用 objName 作为 requestScope 的变量可供视图层调用。
